In [5]:
!pip install deepsparse[yolov8]

  Using cached deepsparse-1.7.1.tar.gz (46.6 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [22 lines of output]
      Loaded version 1.7.1 from C:\Users\hwKIM\AppData\Local\Temp\pip-install-tc6mu_lh\deepsparse_2f547ecd865e49618e1318599f1b2c1e\src\deepsparse\version.py
      Traceback (most recent call last):
        File "C:\Users\hwKIM\anaconda3\envs\dLproj\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "C:\Users\hwKIM\anaconda3\envs\dLproj\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\hwKIM\anaconda3\envs\dLproj\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 118, in get_requires_for_build_wheel
          return hook(config_settings)
       

In [4]:
!yolo task=detect mode=export model=best(9).pt format=onnx opset=13

Ultralytics YOLOv8.2.8 🚀 Python-3.11.9 torch-2.3.0 CPU (13th Gen Intel Core(TM) i5-13420H)
Model summary (fused): 168 layers, 11127906 parameters, 0 gradients

PyTorch: starting from 'best(9).pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 10, 8400) (21.5 MB)
requirements: Ultralytics requirement ['onnx>=1.12.0'] not found, attempting AutoUpdate...






   ---------------------------------------- 0.0/14.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.4 MB 1.3 MB/s eta 0:00:12
    --------------------------------------- 0.2/14.4 MB 2.3 MB/s eta 0:00:07
   - -------------------------------------- 0.6/14.4 MB 5.1 MB/s eta 0:00:03
   --- ------------------------------------ 1.1/14.4 MB 6.3 MB/s eta 0:00:03
   ---- ----------------------------------- 1.5/14.4 MB 6.9 MB/s eta 0:00:02
   ----- ---------------------------------- 1.9/14.4 MB 6.9 MB/s eta 0:00:02
   ------ --------------------------------- 2.5/14.4 MB 7.5 MB/s eta 0:00:02
   --------- --

In [2]:
!yolo export model=best(9).pt format=openvino

Ultralytics YOLOv8.2.8 🚀 Python-3.11.9 torch-2.3.0 CPU (13th Gen Intel Core(TM) i5-13420H)
Model summary (fused): 168 layers, 11127906 parameters, 0 gradients

PyTorch: starting from 'best(9).pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 10, 8400) (21.5 MB)

OpenVINO: starting export with openvino 2024.1.0-15008-f4afc983258-releases/2024/1...
OpenVINO: export success ✅ 3.7s, saved as 'best(9)_openvino_model\' (42.8 MB)

Export complete (11.6s)
Results saved to C:\D\DL_proj\rngus
Predict:         yolo predict task=detect model=best(9)_openvino_model imgsz=640  
Validate:        yolo val task=detect model=best(9)_openvino_model imgsz=640 data=data.yaml  
Visualize:       https://netron.app
💡 Learn more at https://docs.ultralytics.com/modes/export


In [8]:
#from deepsparse import Pipeline
from collections import defaultdict

import cv2
import numpy as np
from ultralytics import YOLO

# Load the YOLOv8 model
model_path = "best(9).onnx"

#yolo_pipeline = Pipeline.create(task="yolov8", model_path=model_path)
model = YOLO(model_path)
cap = cv2.VideoCapture(0)

TARGET_WIDTH = 640
TARGET_HEIGHT = 480
cap.set(cv2.CAP_PROP_FRAME_WIDTH, TARGET_WIDTH)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, TARGET_HEIGHT)

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model(frame)

        # Get the boxes and track IDs
        boxes = results[0].boxes.xywh.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Plot the tracks
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y)))  # x, y center point
            if len(track) > 30:  # retain 90 tracks for 90 frames
                track.pop(0)

            # Draw the tracking lines
            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)

        # Display the annotated frame
        cv2.imshow("YOLOv8 Tracking", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()

WARNING  Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading best(9).onnx for ONNX Runtime inference...
requirements: Ultralytics requirement ['onnxruntime'] not found, attempting AutoUpdate...
   ---------------------------------------- 5.6/5.6 MB 9.7 MB/s eta 0:00:00
   ---------------------------------------- 46.0/46.0 kB ? eta 0:00:00
   ---------------------------------------- 86.8/86.8 kB 5.1 MB/s eta 0:00:00
   ---------------------------------------- 95.2/95.2 kB ? eta 0:00:00

requirements: AutoUpdate success  7.1s, installed 1 package: ['onnxruntime']
requirements:  Restart runtime or rerun command for updates to take effect


0: 640x640 1 Face, 1 Leye open, 1 Reye open, 1 Mouth, 184.7ms
Speed: 14.9ms preprocess, 184.7ms inference, 1466.9ms postprocess per image at shape (1, 3, 640, 640)


AttributeError: 'NoneType' object has no attribute 'int'

In [ ]:
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO("best(9).pt")

# Export the model to NCNN format
model.export(format="ncnn")